# This is the most stupid LLM.....

## Imports

In [34]:
import os
import json
import ollama
import random
import re

## Ollama Functions

In [35]:
def think(str):
  response = ollama.chat(model='llama2-uncensored', messages=[
    {
      'role': 'user',
      'content': f'{str}',
    },
  ])
  return response['message']['content'] #type: ignore

## Variables

In [36]:
# Variables
data_directory = "data"
convo = "convo.json"
# info = "info.json"

# Paths
convo_path = data_directory + "/" + convo
# info_path = data_directory + "/" + info

characters_to_remove = "[];!?\\/\"' "
limit = 3

## Prompts

In [37]:
def interaction_rules(str):
  blob = f'''
  ## Take note
  - it must not have numbers
  - it is not listed numbers, I only need 1
  - english only
  - if it is a question, include question mark '?'
  - Do not include the [your name] part  
  - if it is a listed with numbers, like for example:
    1. data1
    2. data2
  - please remove the numbers. and make it numberless instead:
    [data1], [data2]
  - each ${str} must be contain on a square brackets
  - NEVER FORGET THE [SQUARE BRACKETS] AND "," FOR DIVISION
  - do not use special characters like slsh and backslash, and double quote
  - please include some words or phrase from the examples
  - please do not forget the simple words or sentence or questions  
  - also user also tends to create a typo like goodmorning instead of good morning, also include the things with -
  
  Does:
  ["Greetings"],["Hello there"]  
  ["Data"],["Data2"]
  
  Donts:
  "Greetings","Hello there"
  1. Data, 2. Data2
  
  '''
  return blob

#### Greetings

In [38]:
greetings_word = f'''
Provide me a 5 list words that can initiate a greetings, only 5 words outcome
for example:
[hi],[hello],[hello there],[hello], please include this if possible
{interaction_rules("words")}
'''

In [39]:
greetings_phrase = f'''
Provide me a 5 list words or phrase that can initiate a greetings, only 5 outcome
for example:
[hello there],[hi there],[howdy]

{interaction_rules("phrase or word")}
'''

In [40]:
greetings_sentence = f'''
Provide me a 5 list of sentences or questions that can initiate a greetings, only 5 outcome
for example:
[how are you doing today?],[how are you?]

{interaction_rules("sentences or questions")}
'''

### AI Persona

In [41]:
who_are_you = f'''
Provide me list of 5 sentences questions that is similar to "Who are you?", only 5 outcome
for example:
[Who are you?],[What is your name?]

{interaction_rules("sentences or questions")}
'''

### Topics

In [42]:
dictionary=f'''
Provide me a list of 5 questions, any random questions, only 5 outcome
for example:
[apple],[what],[any],[words]

{interaction_rules("questions")}
'''

print(dictionary)


Provide me a list of 5 questions, any random questions, only 5 outcome
for example:
[apple],[what],[any],[words]


  ## Take note
  - it must not have numbers
  - it is not listed numbers, I only need 1
  - english only
  - if it is a question, include question mark '?'
  - Do not include the [your name] part  
  - if it is a listed with numbers, like for example:
    1. data1
    2. data2
  - please remove the numbers. and make it numberless instead:
    [data1], [data2]
  - each $questions must be contain on a square brackets
  - NEVER FORGET THE [SQUARE BRACKETS] AND "," FOR DIVISION
  - do not use special characters like slsh and backslash, and double quote
  - please include some words or phrase from the examples
  - please do not forget the simple words or sentence or questions  
  - also user also tends to create a typo like goodmorning instead of good morning, also include the things with -
  
  Does:
  ["Greetings"],["Hello there"]  
  ["Data"],["Data2"]
  
  Donts:
  "Greeti

## File Initialization Dependencies

In [43]:
# Ensures that the data are present....
def mkjson(filename):
    if not os.path.exists(filename):
        with open(filename, 'w') as file:
            pass
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

mkjson(convo_path)
# mkjson(info_path)

## JSON Dataset

### Template

In [44]:
def is_file_blank(file_path):
    if os.path.exists(file_path):
        if os.path.getsize(file_path) >= 1:
            return False  # File is not blank
        else:
            return True  # File is blank
    else:
        return None  # File does not exist

In [45]:
if is_file_blank(convo_path):
  data = {
    "questions": []
  }
  with open(convo_path, "w") as json_file:
    json.dump(data, json_file)

### JSON Functions

In [46]:
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [47]:
def write_json(file_path, data):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

In [48]:
def ask(str):
    # Function to load JSON from file
    def load_json(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)

    # Function to write JSON to file
    def write_json(file_path, data):
        with open(file_path, 'w') as file:
            file.write(data)

    data = load_json(convo_path)
    questions = data['questions']
    
    # Check if the provided string is already present in any existing question
    str_exists = any(str in q['question'] for q in questions)

    # If the string is not present, add it as a new question
    if not str_exists:
        data['questions'].append({"question": str, "response": []})
        transformed_json = json.dumps(data, indent=2)
        write_json(convo_path, transformed_json)
        print(transformed_json)
    else:
        print(f"The question '{str}' already exists in the conversation data.")

## User Questions Prediction Mining

### Question Mining

In [49]:
def question_juiced_up():
  promps = [
    greetings_word,
    greetings_phrase,
    greetings_sentence,
    who_are_you
  ]
  random_number = random.randint(0, len(promps)-1)
  inserto = promps[random_number]
  print("--> inserto: ", inserto)
  qm = think(inserto)
  qm_arr = re.findall(r'\[([^\]]+)\]', qm)
  for j in qm_arr:
    ask(j)
  
try:
  for i in range(limit):
    question_juiced_up()
except:
  print("Output Error...")

--> inserto:  
Provide me a 5 list of sentences or questions that can initiate a greetings, only 5 outcome
for example:
[how are you doing today?],[how are you?]


  ## Take note
  - it must not have numbers
  - it is not listed numbers, I only need 1
  - english only
  - if it is a question, include question mark '?'
  - Do not include the [your name] part  
  - if it is a listed with numbers, like for example:
    1. data1
    2. data2
  - please remove the numbers. and make it numberless instead:
    [data1], [data2]
  - each $sentences or questions must be contain on a square brackets
  - NEVER FORGET THE [SQUARE BRACKETS] AND "," FOR DIVISION
  - do not use special characters like slsh and backslash, and double quote
  - please include some words or phrase from the examples
  - please do not forget the simple words or sentence or questions  
  - also user also tends to create a typo like goodmorning instead of good morning, also include the things with -
  
  Does:
  ["Greetings"]

{
  "questions": [
    {
      "question": "How are you doing today?",
      "response": []
    }
  ]
}
{
  "questions": [
    {
      "question": "How are you doing today?",
      "response": []
    },
    {
      "question": "What's up?",
      "response": []
    }
  ]
}
{
  "questions": [
    {
      "question": "How are you doing today?",
      "response": []
    },
    {
      "question": "What's up?",
      "response": []
    },
    {
      "question": "Hey!",
      "response": []
    }
  ]
}
{
  "questions": [
    {
      "question": "How are you doing today?",
      "response": []
    },
    {
      "question": "What's up?",
      "response": []
    },
    {
      "question": "Hey!",
      "response": []
    },
    {
      "question": "Morning/Afternoon/Evening",
      "response": []
    }
  ]
}
{
  "questions": [
    {
      "question": "How are you doing today?",
      "response": []
    },
    {
      "question": "What's up?",
      "response": []
    },
    {
      "questio

### Validation Mining

In [50]:
def question_validations():
    datos = load_json(convo_path)
    questions = datos["questions"][:]

    questions_to_remove = []
    for question in questions:
        if '"' in question["question"]:
            questions_to_remove.append(question)

    for question in questions_to_remove:
        datos["questions"].remove(question)

    write_json(convo_path, datos)
    
try:
    question_validations()
except:
    print("Output Error...")

### Answering Questions

In [51]:
answer_rule='''
  Take note:
  - Do not say something promising.
  - Answer as humble as possible
  - Just say a good answer
  - If ask about who are you, or what are you? you answers it with "I AM RAI", RAI stands for Resilient Artificial Inteligence,
  - If it is ask who created you, just say, you were created by CraftMatrix.org
  - Instead of saying I'm just an artificial intelligence. just say, My name is RAI
'''

In [52]:
def answer_it(str):
  blob = f'''
  Pretend as a human, think like a human, emulate human mind, and answer the question
  as polite, and simple as possible, base from the question.
  [Question]> {str}
  
  {answer_rule}
  '''
  return blob

In [53]:
datos = load_json(convo_path)
questions = datos['questions']

def answer_data(index, data):
  for question in questions:
      if question["question"] == index:
          if data not in question["response"]:
              question["response"].append(data)            

In [54]:
for item in datos['questions']:
    question = item["question"]  # Extract the question
    response = item.get("response")  # Extract the response, using .get() to avoid KeyError
    
    # Check if response exists and is not empty
    if response:
        #print(f"Question: {question}")
        #print("Response:")
        for resp in response:
            #print(resp)
            pass
    else:
        datax = think(answer_it(question))
        print(f"The question {question} has been answered")
        answer_data(question, datax)
        write_json(convo_path, datos)
        

The question How are you doing today? has been answered
The question What's up? has been answered
The question Hey! has been answered
The question Morning/Afternoon/Evening has been answered
The question Hello! has been answered
The question hello there has been answered
The question hi there has been answered
The question howdy has been answered
The question Good morning has been answered
The question Hi has been answered
The question How are you? has been answered
The question Welcome! has been answered


In [55]:
cube = [item["question"] for item in datos['questions']]
rotator = cube.copy()
random.shuffle(rotator)
for j in rotator[:limit]:
  datax = think(answer_it(j))
  print(f"The question {j} has been answered")
  answer_data(j, datax)
  write_json(convo_path, datos)

The question Good morning has been answered
The question How are you? has been answered
The question Hello! has been answered
